In [26]:
from langgraph.graph import StateGraph,START,END
from langchain_openai import ChatOpenAI
from typing import TypedDict,Literal,Annotated
from langchain_core.messages import BaseMessage,HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph.message import add_messages
from dotenv import load_dotenv

In [27]:
load_dotenv()

True

In [28]:
class ChatbotState(TypedDict):
    messages: Annotated[list[BaseMessage],add_messages]

In [29]:
llm = ChatOpenAI(model="gpt-4o-mini")

def chat_node(state:ChatbotState):
    
    response = llm.invoke(state["messages"])
    return {"messages":[response]}


In [31]:
# add checkpointer to utilize local memory for chat history
checkpointer = MemorySaver()
# create graph with nodes and edges
graph = StateGraph(ChatbotState)

graph.add_node("chat_node",chat_node)

graph.add_edge(START,"chat_node")
graph.add_edge("chat_node",END)
# compile graph to create chatbot object
chatbot = graph.compile(checkpointer=checkpointer)

In [38]:
# create a thread id for chat history
thread_id = "1"
# start chatting with AI chatbot
while True:
    user_query = input("type here:")
    print(f"User: {user_query}")
    
    # loop exit criteria
    if user_query.strip().lower() in ['exit','bye','end','quit']:
        break 

    #create config for memory allocation to chat history
    CONFIG = {'configurable':{'thread_id':thread_id}}
    #response = chatbot.invoke({"messages": [HumanMessage(content=user_query)]},config=config)
    print(f"AI: ")
    for msg_chunk,metadata in chatbot.stream({'messages':HumanMessage(content=user_query)},
                                             config=CONFIG,
                                             stream_mode="messages"):
        if msg_chunk.content:
            print(msg_chunk.content,end="",flush=True)
    print(end='\n')
    


User: Hi
AI: 
Hello! How can I help you today?
User: write a brief summary on Artificial Intelligence
AI: 
Artificial Intelligence (AI) is a branch of computer science focused on creating systems that can perform tasks typically requiring human intelligence. This includes abilities such as learning, reasoning, problem-solving, understanding natural language, and recognizing patterns. AI encompasses various subfields, including machine learning, where algorithms improve through experience, and deep learning, which mimics human brain function to analyze complex data.

AI applications are widespread, impacting industries like healthcare (for diagnostics and treatment recommendations), finance (for fraud detection and algorithmic trading), transportation (in the development of autonomous vehicles), and entertainment (through personalized content recommendations). While AI offers significant benefits, it also raises ethical and societal concerns, including issues of bias, job displacement, 